In [19]:
import pandas as pd
import math
from random import randint, seed

import warnings
warnings.filterwarnings("ignore")

In [20]:
DATA = 'data'
seed(42)

# New sintetic data

In [139]:
#desease = pd.read_csv(f'{DATA}/sintetic_deseases.csv', delimiter=';') sasha directory
desease = pd.read_csv('new_data.csv', delimiter=';')
desease = desease[desease['Column1']!='Болезнь']
desease = desease.rename(columns={
    'Column1': 'Диагноз',
    'Column2': 'Направление',
    'Column3': 'Обязательность',
    'Column4': 'Врач'
})
desease = desease[desease['Обязательность'].isin(('Обязательно', 'По показанию'))]

In [140]:
def format_doctor_name(doctor_name):
    if 'Врач-' in doctor_name:
        subsample = doctor_name.split('-')[1]
        subsample = subsample[0].upper() + subsample[1:]
        return subsample
    elif doctor_name == 'ЛОР-врач':
        return 'Отоларинголог'
    else: 
        return doctor_name

In [141]:
desease['Врач'] = desease['Врач'].map(format_doctor_name)

In [142]:
desease

,Диагноз,Направление,Обязательность,Врач
1,Астма,Спирометрия,Обязательно,Пульмонолог
2,Астма,Пикфлоуметрия,По показанию,Пульмонолог
3,Астма,Аллергические анализы,По показанию,Аллерголог
4,Астма,Мониторинг показателей физической активности,По показанию,Пульмонолог
5,Астма,Консультация пульмонолога,Обязательно,Пульмонолог
...,...,...,...,...
782,Сахарный диабет,Консультация эндокринолога,Обязательно,Эндокринолог
783,Артериальная гипертензия,Клинический осмотр,Обязательно,Врач общей практики
784,Артериальная гипертензия,Измерение артериального давления,Обязательно,Медсестра
785,Артериальная гипертензия,Общий анализ крови,Обязательно,Лаборант


In [143]:
desease['Врач'].unique()

array(['Пульмонолог', 'Аллерголог', 'Ортопед', 'Ревматолог',
       'Эндокринолог', 'Психиатр', 'Психолог', 'Терапевт',
       'УЗИ-специалист', 'Гематолог', 'Инфекционист', 'Отоларинголог',
       'Гастроэнтеролог', 'Невролог', 'Онколог', 'Иммунолог', 'Проктолог',
       'Дерматолог', 'Офтальмолог', 'Кардиолог', 'Педиатр', 'Гинеколог',
       'Генетик', 'Нефролог', 'Клинический лаборант',
       'Травматолог-ортопед', 'Логопед', 'Уролог', 'Врач общей практики',
       'Хирург', 'Нейрохирург', 'Миколог', 'Радиолог', 'Лаборант',
       'Инфекционист ', 'Специалист по электрофизиологии', 'Реабилитолог',
       'Врач реабилитолог', 'Рентгенолог', 'Ветеринар', 'Микробиолог',
       'Неврохирург', 'Патологоанатом', 'Оториноларинголог',
       'Физкультурник', 'Физиолог', 'Маммолог', 'Фтизиатр',
       'Колопроктолог', 'Психотерапевт', 'Вирусолог', 'Бактериолог',
       'Медсестра'], dtype=object)

In [144]:
doctors_freq = {}
for i in desease['Врач'].unique():
    doctors_freq[i] = {'n' : desease[desease['Врач']==i].shape[0],
                       'ids' : []
                      }
doctors_freq

{'Пульмонолог': {'n': 32, 'ids': []},
 'Аллерголог': {'n': 12, 'ids': []},
 'Ортопед': {'n': 18, 'ids': []},
 'Ревматолог': {'n': 18, 'ids': []},
 'Эндокринолог': {'n': 40, 'ids': []},
 'Психиатр': {'n': 30, 'ids': []},
 'Психолог': {'n': 12, 'ids': []},
 'Терапевт': {'n': 10, 'ids': []},
 'УЗИ-специалист': {'n': 10, 'ids': []},
 'Гематолог': {'n': 12, 'ids': []},
 'Инфекционист': {'n': 47, 'ids': []},
 'Отоларинголог': {'n': 9, 'ids': []},
 'Гастроэнтеролог': {'n': 80, 'ids': []},
 'Невролог': {'n': 99, 'ids': []},
 'Онколог': {'n': 31, 'ids': []},
 'Иммунолог': {'n': 5, 'ids': []},
 'Проктолог': {'n': 1, 'ids': []},
 'Дерматолог': {'n': 71, 'ids': []},
 'Офтальмолог': {'n': 9, 'ids': []},
 'Кардиолог': {'n': 29, 'ids': []},
 'Педиатр': {'n': 14, 'ids': []},
 'Гинеколог': {'n': 14, 'ids': []},
 'Генетик': {'n': 25, 'ids': []},
 'Нефролог': {'n': 2, 'ids': []},
 'Клинический лаборант': {'n': 1, 'ids': []},
 'Травматолог-ортопед': {'n': 4, 'ids': []},
 'Логопед': {'n': 1, 'ids': []},
 '

# Generate visits DataFrame

In [175]:
visits = pd.DataFrame(columns=desease.columns)

In [176]:
all_referrals = desease['Направление'].unique()
all_mandory = ['Обязательно', 'По показанию']

In [177]:
for doctor_i in list(desease['Врач'].unique()):
    doctor_info = desease[desease['Врач']==doctor_i]
    for desease_i in doctor_info['Диагноз'].unique():
        desease_info = doctor_info[doctor_info['Диагноз']==desease_i]
        true_referrals = desease_info.sample(desease_info.shape[0]-1)
        false_referrals = pd.DataFrame(columns=desease.columns)
        for referral_i in range(randint(0, 3)): # генерация неправильных направлеий
            new_referral = {
            'Диагноз': desease_i,
            'Направление': all_referrals[randint(1, len(all_referrals)-1)],
            'Обязательность': all_mandory[randint(0, 1)],
            'Врач': doctor_i,
            }
            false_referrals = pd.concat([false_referrals, pd.DataFrame(data=new_referral, index=[0], columns=desease.columns)])
        new_samples = pd.concat([true_referrals, false_referrals]).reset_index()
        new_samples = new_samples.drop(columns=['index'])
        visits = pd.concat([visits, new_samples])
visits = visits.reset_index().drop(columns=['index'])
print(visits.shape)
visits.head()

(979, 5)


,Диагноз,Направление,Обязательность,Врач,Диагноз и Направление
0,Астма,Мониторинг показателей физической активности,По показанию,Пульмонолог,"(Астма, Мониторинг показателей физической акти..."
1,Астма,Пикфлоуметрия,По показанию,Пульмонолог,"(Астма, Пикфлоуметрия)"
2,Астма,Спирометрия,Обязательно,Пульмонолог,"(Астма, Спирометрия)"
3,Астма,Функциональная эзофагогастродуоденоскопия (ФЭГДС),По показанию,Пульмонолог,NaN
4,Астма,Нейрологическое обследование,По показанию,Пульмонолог,NaN


In [178]:
desease['Диагноз и Направление'] = desease[['Диагноз', 'Направление']].apply(tuple, axis=1)

## Some tests to check generation 

In [179]:
true_pairs = list(zip(list(desease['Диагноз']), list(desease['Направление'])))
true_and_false_pairs = list(zip(list(visits['Диагноз']), list(visits['Направление'])))
the_same = len(list(set(true_pairs) & set(true_and_false_pairs)))

print(f'всего правильных пар болезней и направлений {desease.shape[0]}')
print(f'в нашем датасете таких пар встречается {the_same}')

всего правильных пар болезней и направлений 786
в нашем датасете таких пар встречается 381


In [180]:
counter = 0
for pair in true_and_false_pairs:
    if pair in true_pairs:
        counter += 1
print(f'корректных назначений для болезней {counter} из {len(true_and_false_pairs)}')

корректных назначений для болезней 393 из 979


# Set ids for doctors

In [181]:
max_id = 1
for doctor in doctors_freq.keys():
    for i in range(math.ceil(doctors_freq[doctor]['n']/5)):
        doctors_freq[doctor]['ids'].append(max_id)
        max_id += 1

In [182]:
def set_id(doctor_name):
    possible_ids = doctors_freq[doctor_name]['ids']
    return randint(min(possible_ids), max(possible_ids))

In [183]:
visits['doctor_id'] = visits['Врач'].map(set_id)

In [184]:
visits['Диагноз и Направление'] = visits[['Диагноз', 'Направление']].apply(tuple, axis=1)

In [185]:
visits.head()

,Диагноз,Направление,Обязательность,Врач,Диагноз и Направление,doctor_id
0,Астма,Мониторинг показателей физической активности,По показанию,Пульмонолог,"(Астма, Мониторинг показателей физической акти...",2
1,Астма,Пикфлоуметрия,По показанию,Пульмонолог,"(Астма, Пикфлоуметрия)",7
2,Астма,Спирометрия,Обязательно,Пульмонолог,"(Астма, Спирометрия)",1
3,Астма,Функциональная эзофагогастродуоденоскопия (ФЭГДС),По показанию,Пульмонолог,"(Астма, Функциональная эзофагогастродуоденоско...",4
4,Астма,Нейрологическое обследование,По показанию,Пульмонолог,"(Астма, Нейрологическое обследование)",5


# Rate doctors

In [186]:
unique_ids = visits['doctor_id'].unique()
doctor_stats = {}
for doctor_id in unique_ids:
    test_doctor = visits[visits['doctor_id']==doctor_id]
    nesses = test_doctor[test_doctor['Обязательность']=='Обязательно']
    unnesses = test_doctor[test_doctor['Обязательность']=='По показанию']
    #print(type(doctor_id))
    print(test_doctor[test_doctor['doctor_id']==1])
    if doctor_id == 2:
        break
    true_nesses = 0
    all_nesses = 0
    for i in list(nesses['Диагноз и Направление']):
        nesses_right_healing = list(desease[desease['Обязательность']=='Обязательно'][desease['Диагноз']==i[0]]['Диагноз и Направление'])
        if i in nesses_right_healing:
            true_nesses += 1 
        all_nesses += len(nesses_right_healing)

    true_unnesses = 0
    all_unnesses = 0
    for i in list(unnesses['Диагноз и Направление']):
        unnesses_right_healing = list(desease[desease['Обязательность']=='По показанию'][desease['Диагноз']==i[0]]['Диагноз и Направление'])
        # print(unnesses_right_healing)
        if i in unnesses_right_healing:
            true_unnesses += 1 
        all_unnesses += len(unnesses_right_healing)

Empty DataFrame
Columns: [Диагноз, Направление, Обязательность, Врач, Диагноз и Направление, doctor_id]
Index: []


In [187]:
all_unnesses

10

### Подсчет для одного доктора

In [188]:
unique_ids = visits['doctor_id'].unique()
doctor_stats = {}
for doctor_id in unique_ids:
    test_doctor = visits[visits['doctor_id']==doctor_id]
    nesses = test_doctor[test_doctor['Обязательность']=='Обязательно']
    unnesses = test_doctor[test_doctor['Обязательность']=='По показанию']
    
    

In [200]:
doctor_1 = visits[visits['doctor_id']==2]

In [201]:
doctor_1.head()

,Диагноз,Направление,Обязательность,Врач,Диагноз и Направление,doctor_id
0,Астма,Мониторинг показателей физической активности,По показанию,Пульмонолог,"(Астма, Мониторинг показателей физической акти...",2
7,Аспергиллома,Лечебная физкультура,По показанию,Пульмонолог,"(Аспергиллома, Лечебная физкультура)",2
14,Идиопатическая неспецифическая интерстициальна...,Высокоразрешающая компьютерная томография груд...,Обязательно,Пульмонолог,(Идиопатическая неспецифическая интерстициальн...,2
18,Кавернозный туберкулёз,Манту-тест или кожная проба Туберкулина,По показанию,Пульмонолог,"(Кавернозный туберкулёз, Манту-тест или кожная...",2
22,Криптогенная организующаяся пневмония,Выделение бронхиального секрета для исследования,Обязательно,Пульмонолог,"(Криптогенная организующаяся пневмония, Выделе...",2


In [190]:
y_true_nesses = [0]*len(desease)
y_pred_nesses = [0]*len(desease)
y_true_unnesses = [0]*len(desease)
y_pred_unnesses = [0]*len(desease)

#заполнение y_true_nesses
for i in range(len(desease)):
    for k in range(len(nesses)):
        if nesses['Диагноз'].iloc[k] == desease['Диагноз'].iloc[i]:
            y_true_nesses[i] = 1

#заполнение y_pred_nesses
for i in range(len(desease)):
    for k in range(len(nesses)):
        if nesses['Диагноз'].iloc[k] == desease['Диагноз'].iloc[i] and nesses['Направление'].iloc[k] == desease['Направление'].iloc[i]:
            #print(desease['Диагноз'].iloc[i], desease['Направление'].iloc[i])
            y_pred_nesses[i] = 1

#заполнение y_true_unnesses
for i in range(len(desease)):
    for k in range(len(unnesses)):
        if unnesses['Диагноз'].iloc[k] == desease['Диагноз'].iloc[i]:
            y_true_unnesses[i] = 1

#заполнение y_pred_unnesses
for i in range(len(desease)):
    for k in range(len(unnesses)):
        if unnesses['Диагноз'].iloc[k] == desease['Диагноз'].iloc[i] and unnesses['Направление'].iloc[k] == desease['Направление'].iloc[i]:
            #print(desease['Диагноз'].iloc[i], desease['Направление'].iloc[i])
            y_pred_unnesses[i] = 1

In [191]:
from sklearn.metrics import fbeta_score
from sklearn.metrics import f1_score

In [192]:
nesses_score = fbeta_score(y_true_nesses, y_pred_nesses, average='macro', beta=2)
unnesses_score = fbeta_score(y_true_unnesses, y_pred_unnesses, average='macro', beta=0.5)

In [197]:
unnesses_score

1.0

In [195]:
SCORE = (5*nesses_score + unnesses_score)/6

In [196]:
SCORE

0.5829075114971894

In [216]:
def count_score_doctor(visits: pd.DataFrame) -> dict:
    unique_ids = visits['doctor_id'].unique()
    doctor_stats = {}
    for doctor_id in unique_ids:
        test_doctor = visits[visits['doctor_id']==doctor_id]
        nesses = test_doctor[test_doctor['Обязательность']=='Обязательно']
        unnesses = test_doctor[test_doctor['Обязательность']=='По показанию']
        #doctor = visits[visits['doctor_id']==doctor_id]

        y_true_nesses = [0]*len(desease)
        y_pred_nesses = [0]*len(desease)
        y_true_unnesses = [0]*len(desease)
        y_pred_unnesses = [0]*len(desease)
        
        #заполнение y_true_nesses
        for i in range(len(desease)):
            for k in range(len(nesses)):
                if nesses['Диагноз'].iloc[k] == desease['Диагноз'].iloc[i]:
                    y_true_nesses[i] = 1
        
        #заполнение y_pred_nesses
        for i in range(len(desease)):
            for k in range(len(nesses)):
                if nesses['Диагноз'].iloc[k] == desease['Диагноз'].iloc[i] and nesses['Направление'].iloc[k] == desease['Направление'].iloc[i]:
                    #print(desease['Диагноз'].iloc[i], desease['Направление'].iloc[i])
                    y_pred_nesses[i] = 1
        
        #заполнение y_true_unnesses
        for i in range(len(desease)):
            for k in range(len(unnesses)):
                if unnesses['Диагноз'].iloc[k] == desease['Диагноз'].iloc[i]:
                    y_true_unnesses[i] = 1
        
        #заполнение y_pred_unnesses
        for i in range(len(desease)):
            for k in range(len(unnesses)):
                if unnesses['Диагноз'].iloc[k] == desease['Диагноз'].iloc[i] and unnesses['Направление'].iloc[k] == desease['Направление'].iloc[i]:
                    #print(desease['Диагноз'].iloc[i], desease['Направление'].iloc[i])
                    y_pred_unnesses[i] = 1
                    
        nesses_score = fbeta_score(y_true_nesses, y_pred_nesses, average='macro', beta=2)
        unnesses_score = fbeta_score(y_true_unnesses, y_pred_unnesses, average='micro', beta=0.5)
        SCORE = (5*nesses_score + unnesses_score)/6*100
        doctor_stats[doctor_id] = SCORE
    return doctor_stats

In [217]:
count_score_doctor(visits)

{2: 67.97480010643245,
 7: 59.770732056513275,
 1: 64.48013648528838,
 4: 64.35291002218406,
 5: 59.674258732302796,
 6: 78.02388547511099,
 3: 70.8058651026393,
 10: 71.6322577044583,
 9: 65.78519357850494,
 8: 99.82188295165393,
 13: 69.33020658363819,
 12: 78.51580552020232,
 11: 63.27272791400157,
 14: 78.10022135297359,
 18: 67.21830432177077,
 17: 99.77099236641222,
 15: 74.941255193452,
 16: 68.47573776024228,
 20: 71.43054201996269,
 26: 68.0078804748195,
 22: 75.2974892901441,
 19: 65.41718054336941,
 25: 67.25062906067049,
 21: 59.826930488028,
 24: 67.91064698466886,
 23: 63.94281451838933,
 29: 71.8948431652237,
 28: 65.93549711267188,
 32: 61.85038091420465,
 31: 63.58658042169723,
 27: 74.04011532296792,
 30: 63.44621549737337,
 34: 69.1107161929893,
 35: 65.03586796070404,
 33: 59.6427193102952,
 37: 59.867016033537126,
 36: 66.58057611143863,
 38: 59.72822010375164,
 39: 59.46769133054766,
 40: 69.4574330467425,
 42: 74.09100590820967,
 41: 82.09999115398826,
 48: 71.31

In [ ]:
desease.to_csv()